In [1]:
import numpy as np
import torch

from torch.utils.data import Dataset

import torch.nn.functional as F
import torch.nn as nn

import matplotlib.pyplot as plt

import myModelinTF

from torch.optim.lr_scheduler import StepLR
import pandas as pd
import tensorflow as tf
from torch.autograd import Variable 

from statistics import mean 

In [2]:
demo = np.load('dataset_train_firsts.npy', allow_pickle=True)
print("Train firsts set size: " + str(len(demo)))

Train firsts set size: 495


In [3]:
class SeriesDataset(Dataset):

    def __init__(self, file):
        self.demo = np.load(file, allow_pickle=True)

    #len(usd)
    def __len__(self):
        return len(self.demo)

    # a_list[1] --> a__list.__getitem__(1)
    def __getitem__(self, index):
        run =self.demo[index]
        states = np.array(run[0]).T
        #states_init1 = states[1,:]
        if states.size == 0:
            print('no states in submatrix')

        actions = np.array(run[1])[:, None]

        action_count = np.array([actions[0][0], len(actions)])
        #actions_counts_array_padded = np.pad(actions_counts_array, ((0,30-len(actions_counts_array)),(0,0)), 'constant').astype('int32') 
        #states_init = np.zeros((len(actions_counts_array_padded), 10)) + states_init1
        #states_init = states[0:len(actions_counts_array),:]
       
        return states, action_count.astype('int32') 

In [4]:
def permute_them(x):
  return torch.permute(x, (1, 0, 2))

In [5]:
def fill_four_with_last(arr):
    prev = np.arange(len(arr))
    prev[arr == 4] = 0
    prev = np.maximum.accumulate(prev)

    # it can be that the first element is 4, so remove those
    temp = arr[prev]
    new_deleted = np.delete(temp, np.where(temp == 4))
    count = len(temp) - len(new_deleted)
    new = np.insert(new_deleted, 0, np.repeat(new_deleted[0], count), axis=0)
    return new

In [130]:
class PG(nn.Module):
    def __init__(self, n_actions):
        super().__init__()
        self.n_actions = n_actions

        # Policy takes input initial conditions and outputs planning route, open-loop control
        self.actor_model = nn.Sequential(
            nn.LSTM(input_size = 10, hidden_size = self.n_actions, num_layers = 1, dropout = 0.1,batch_first = True),
        )

        self.logprobs = []
        self.state_values = []
        self.rewards = []

    def forward(self, states):
        logits, (h_T, c_T)  = self.actor_model(states)
        return logits[-1]
    # Run agent in environment to create sample trajectories by generator
    # The environment model is a seq2seq model
    def generate_session(self, sysmodel, init_states, traj_idx,  t_max=50):
        norm_params = pd.read_csv('D:\\units\\thesis\\thesis\\train_reward_inverse_rl_w_sys_model\\norm_params.csv', header = None).to_numpy().T

        states, actions, actions_probs_policy, rewards = [], [], [], []
        states_init_idx = init_states[traj_idx]

        states = torch.FloatTensor(states_init_idx[0].T)
        states = states.unsqueeze(0)
        states_history = states[:,-1,:].unsqueeze(0)
        list_actions_taken = []
        actions_probs_policy_taken = []
        action = 0
        for t in range(0, t_max, 5):
            logits = policy(states) # forward pass
            logits = logits[-1]
            probs = nn.functional.softmax(logits, -1) # get estimated actions for states
            #log_probs = nn.functional.log_softmax(logits, -1)
            
            actions_probs_policy = np.squeeze(probs).clone().detach().numpy()
            actions_probs_policy_rep = np.tile(actions_probs_policy,(5,1))
            actions_probs_policy_taken.append(actions_probs_policy_rep)
            # actions = []
            # for prob in actions_probs_policy:                  
            #     actions.append(np.random.choice(5,  p = prob))
            action_prev = action
            action = np.random.choice(5,  p = actions_probs_policy)
            if action == 4:
                action = action_prev

            action_rep = np.repeat(action, 5)
            list_actions_taken.append(action_rep)

            #list_actions_filled = np.append(list_actions_filled, a)
            
            # Apply dict to go from 1,2,3,4 as action to [0,2400,0] [0,2400,1000]  [2300,0,0]  [2300,0,1000] 
            my_dict = {0:[0,2400,0], 1:[0,2400,1000], 2:[2300,0,0], 3:[2300,0,1000]}
            #actions_array = np.zeros((500,3))  
            actions_array = map(my_dict.get, np.array(list_actions_taken).flatten())
            actions_array = np.array(list(actions_array))

            actions_array_sysmodel = np.concatenate([np.arange(actions_array.shape[0])[:,None]+1+t*len(list_actions_taken),actions_array], axis=1) 
            actions_array_sysmodel = tf.expand_dims(actions_array_sysmodel, axis = 0, name=None)
            
            states = sysmodel(actions_array_sysmodel)

            # Revert normalized values for states
            states = states*norm_params[:,0]+norm_params[:,1]

            states = torch.FloatTensor(np.array(states))
            # More actions fed to the sysmodel, higher the accuracy, but the resulting states will not be the ones effectively experienced, so history is needed
            # Rewards can be calculated on the set of states experienced with all actions at once, but then the scope is just to evaluate the set of actions in the whole episode, not step by step
            states_history = torch.cat((states_history, states[:,-5:,:]), 1)

            # if done:
            #     break
        
        # print(actions_taken)
        # actions_taken = np.array(actions_taken)
        actions_probs_policy_taken = np.array(actions_probs_policy_taken).reshape(t_max, 5)
        # states_all = sysmodel(actions_taken_reshape)
        # states_all = states_new*norm_params[:,0]+norm_params[:,1]
        rewards = get_rewards(states, np.array(list_actions_taken))
        return states_history, states, np.array(list_actions_taken).flatten(), actions_probs_policy_taken, rewards

In [7]:
class Critic(nn.Module):
    def __init__(self):
        super().__init__()

        self.critic_model = nn.Sequential(
            # nn.Linear(10, 32),
            # nn.ReLU(inplace=False),
            # nn.Linear(32, 1)
            nn.Conv1d(in_channels=1, out_channels=8, kernel_size=5),
            nn.LSTM(input_size = 8, hidden_size = 1, num_layers = 1, dropout = 0.1,batch_first = True),
        )

    def forward(self, state):
        state = torch.FloatTensor(state)
        #state_value = self.critic_model(state)
        logits, (state_value, c_T)  = self.critic_model(state)
        return state_value

In [52]:
def get_y_rewards(model,states, rewards,gamma=0.9):
    states = states.numpy()
    for idx_i in range(len(rewards)):
        for idx in range(idx_i + 1, len(rewards)):
            out2 = model(states[:,idx,:]).squeeze()
            rewards[idx_i] += out2.detach().numpy()
            continue
    return rewards

def get_critic_rewards(model,states, rewards,gamma=0.9):
    G = np.zeros_like(rewards, dtype=float)
    G[-1] = rewards[-1]
    states = states.numpy()
    for idx in range(len(rewards)):
        out1 = model(states[:,idx,:])
        #out2 = model(states[:,idx,:])
        G[idx] = out1.detach().numpy()
    return G

def get_advantage_rewards(model,states, rewards,gamma=0.9):
    G = np.zeros_like(rewards, dtype=float)
    G[-1] = rewards[-1]
    states = states.numpy()
    for idx in range(len(rewards)-1):
        out1 = model(states[:,idx,:])
        out2 = model(states[:,idx+1,:])
        G[idx] = rewards[idx] + out2.detach().numpy() - out1.detach().numpy()
    return G

def get_rewards(states, actions):
    
    states = states.numpy()
    states = states.squeeze()
    rewards = np.zeros((np.shape(states)[0],1))

    actions = actions.flatten()
    # print(actions)
    # max_hotspot =  np.max(states)
    # if max_hotspot < 300:
    #     rewards[-1] = 100

    # if np.where( 229 >= states[:,1].all() <= 231)[0] < 405 & np.where( 229 >= states[:,1].all() <= 231)[0] > 340:
    #     rewards[-1] = rewards[-1] + 100

        
    for idx in range(np.shape(states)[1]):
        idx = idx -1
        # temps = np.array(states[idx])
        # if temps.any() > 230:
        #     rewards[idx] = rewards[idx] -30
        
        # subset = actions[idx-20:]
        # if np.sum(np.diff(subset)) == 0:
        #     rewards[idx] = rewards[idx] + 20

        # subset = temps[idx-10:]
        # if np.sum(np.diff(subset)) < 10:
        #     rewards[idx] = rewards[idx] + 10

        # Save power consumption
        # if actions[idx] == 1 | actions[idx] == 3:
        #     rewards[idx] = rewards[idx] -10

        # Action 4 is encouraged , has the meaning to keep the previous action
        if actions[idx] == 4:
            rewards[idx] = rewards[idx] + 10

        # temps_subset = np.array([states[idx, 1], states[idx-1, 1]])
        # if np.diff(temps_subset) < 0:
        #     rewards[idx] = rewards[idx] - 100


    return rewards

def to_one_hot(y_tensor, ndims):
    y_tensor = y_tensor.type(torch.LongTensor).view(-1, 1)
    y_one_hot = torch.zeros(
        y_tensor.size()[0], ndims).scatter_(1, y_tensor, 1)
    return y_one_hot

In [131]:
# DEVICE
DEVICE = 'cpu'

# ENV SETUP
sysmodel = myModelinTF.load_model()
n_actions = 5

# INITILIZING POLICY AND REWARD FUNCTION
policy = PG(n_actions)
policy.to(DEVICE)

critic = Critic()
critic.to(DEVICE)

norm_params = pd.read_csv('D:\\units\\thesis\\thesis\\train_reward_inverse_rl_w_sys_model\\norm_params.csv', header = None).to_numpy().T


# LOADING EXPERT/DEMO SAMPLES
DATASETNPY = "dataset_train_firsts.npy"

dataset_inits = SeriesDataset(DATASETNPY)

##### TUNE MODEL WITH REWARD LOSS

mean_rewards = []
mean_loss = []
mean_loss_rew = []
EPISODES_TO_PLAY = 3
sample_trajs = []

return_list, sum_of_loss_list = [], []

# Retrain all params
optimizer_policy = torch.optim.Adam(policy.parameters(), 1e-1, weight_decay=1e-2)
optimizer_critic = torch.optim.Adam(critic.parameters(), 1e-1, weight_decay=1e-2)

init_states = np.load(DATASETNPY, allow_pickle=True)

criterion2 = nn.MSELoss()

#Epochs
for i in range(10):
    trajs = [policy.generate_session(sysmodel, init_states, l) for l in range(EPISODES_TO_PLAY)]

    # Episodes
    for traj in trajs:
        states_history, states, actions, probs, rewards = traj

        cumulative_returns_np = np.array(get_advantage_rewards(critic,states,rewards, 0.7)).squeeze()
        cumulative_returns = torch.tensor(cumulative_returns_np, dtype=torch.float32)
        #cumulative_returns = (cumulative_returns - torch.mean(cumulative_returns)) # torch.mean

        actions_tensor = torch.from_numpy(actions)
        probs_tensor = torch.from_numpy(np.array(probs))

        log_probs_for_actions = torch.log(torch.sum(
            probs_tensor * to_one_hot(actions_tensor, 5), dim =1))
    
        loss_policy = -log_probs_for_actions*cumulative_returns_np
        # UPDATING THE POLICY NETWORK
        optimizer_policy.zero_grad()
        loss_policy.requires_grad = True
        loss_policy.sum().backward()
        torch.nn.utils.clip_grad_norm_(policy.parameters(), 0.01)
        optimizer_policy.step()

        loss_critic = criterion2(torch.from_numpy(np.array(get_critic_rewards(critic,states,rewards, 0.7).sum())).to(torch.float32), \
                                 torch.from_numpy(np.array(get_y_rewards(critic, states, rewards, 0.7).sum())).to(torch.float32))
        # UPDATING THE CRITIC NETWORK
        optimizer_critic.zero_grad()
        loss_critic.requires_grad = True
        loss_critic.backward()
        optimizer_critic.step()

        #returns = sum(rewards)
        sum_of_loss = loss_policy.sum().detach().numpy()
        print(f"\nLoss Actor: {sum_of_loss}\n")
        print(f"\nLoss Critic: {loss_critic}\n")
        #print(f"\nCumulative return: {cumulative_returns}\n")
        #return_list.append(returns)
        sum_of_loss_list.append(sum_of_loss)
        states_plot = np.array(states_history.squeeze())

    plt.subplot(2, 2, 1)
    plt.title(f"Loss per {EPISODES_TO_PLAY} episodes")
    plt.plot(sum_of_loss_list)
    plt.grid()

    plt.subplot(2, 2, 2)
    plt.title(f"Rewards")
    plt.plot(rewards)
    plt.grid()

    plt.subplot(2, 2, 3)
    plt.title(f"Activations")
    plt.plot(actions[0:50])
    plt.grid()

    plt.subplot(2, 2, 4)
    plt.title(f"Temperatures")
    plt.plot(states_plot[0:50,:])
    plt.grid()

    # plt.show()
    plt.savefig('GCL_learning_curve.png')
    plt.close()

d:\units\thesis\thesis\train_reward_inverse_rl_w_sys_model\.venv\lib\site-packages\torch\nn\modules\rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "



Loss Actor: 2.2145032348682974


Loss Critic: 6673.67529296875


Loss Actor: -1.2570560409721012


Loss Critic: 39878.71875


Loss Actor: 1.4505519860255671


Loss Critic: 1.7746305465698242


Loss Actor: 0.9308734716519314


Loss Critic: 14256.2998046875


Loss Actor: 1.827767665585331


Loss Critic: 82680.5390625


Loss Actor: 2.854206141407161


Loss Critic: 2898.772705078125


Loss Actor: 1.6882379050181306


Loss Critic: 770.52685546875


Loss Actor: -1.5089543960451044


Loss Critic: 31466.80078125


Loss Actor: 0.9978937702345483


Loss Critic: 26851.2890625


Loss Actor: 0.7623581033107945


Loss Critic: 5492.3994140625


Loss Actor: 0.6895609420682263


Loss Critic: 179641.921875


Loss Actor: 2.2463633891677355


Loss Critic: 17701.22265625


Loss Actor: 1.932627747589272


Loss Critic: 62.10810852050781


Loss Actor: 1.2713062558784782


Loss Critic: 20445.517578125


Loss Actor: 0.8550949646895661


Loss Critic: 2097.982421875


Loss Actor: -0.15105280675441923


Loss Crit